# 5.1

In [11]:
import pandas as pd
import numpy as np
import math
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, RandomTreesEmbedding, VotingClassifier

Все задания выполняются на основе датасета UCI ML Breast Cancer Wisconsin. (https://goo.gl/U2Uwz2)  
Все признаки являются числовыми.

In [13]:
X, y = load_breast_cancer(return_X_y=True)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

1) Реализовать функцию подсчета индекса Джини

In [5]:
def compute_gini_impurity(labels):
    """
    :arg labels: np.array of shape (n_objects,)
    
    :return: gini_impurity: float
    """
    unique, counts = np.unique(labels, return_counts=True)
    res = 1
    for i in range(len(unique)):
        p =  counts[i]/np.sum(counts)
        res = res - p ** 2
    return res

2) Реализовать функцию подсчета энтропии

In [9]:
def compute_entropy(labels):
    """
    :arg labels: np.array of shape (n_objects,)
    
    :return: entropy: float
    """
    unique, counts = np.unique(labels, return_counts=True)
    res = 0
    for i in range(len(unique)):
        p =  counts[i]/np.sum(counts)
        res = res - p*math.log2(p)
    return res

3) Реализовать функцию подсчета Information Gain

In [10]:
def calculate_information_gain(s0, *args):
    """
    Calculates IG for a split s_1, ..., s_k.
        :arg s0: the whole set, sequence of shape (n_objects,)
        :arg args: split — a list of sequences, such that s0 = union(s_1, ..., s_k) and
           for all i, j: intersection(s_i, s_j) = 0
           
        :return: information_gain: float
    """
    unique_s0, counts_s0 = np.unique(s0, return_counts=True)

    unique_args = []
    counts_args = []

    for i in range(len(args)):
        unique_buf, counts_buf = np.unique(args[i], return_counts=True)
        unique_args.append(unique_buf)
        counts_args.append(counts_buf)

    res = compute_entropy(s0)

    for i in range(len(args)):
        res = res - (np.sum(counts_args[i])/np.sum(counts_s0))*compute_entropy(args[i])

    return res

In [11]:
calculate_information_gain([1, 2, 3, 4, 5, 6], [1, 2, 3, 4], [5, 6])

0.9182958340544896

## Задачи с проверкой ответа

В следующих заданих требуется так или иначе изменить параметры дерева решений и оценить результат.  
Как метрику качества будем использовать F1-score. В каждом задании необходимо поменять __только один__ параметр.

In [12]:
def compute_metric(clf):
    np.random.seed(42)
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred), 6)

Посчитаем значение метрики со стандартными параметрами.

In [13]:
compute_metric(DecisionTreeClassifier())

0.957746

4) Добавьте следующее условие: для выполнения сплита должно быть как минимум 22 объекта. Чему равна метрика F1 на отложенной выборке?

In [14]:
compute_metric(DecisionTreeClassifier(min_samples_split=22))

0.951049

5) Добавьте следующее условие: в листе должно быть как минимум 10 элементов. Чему равна метрика F1 на отложенной выборке?

In [15]:
compute_metric(DecisionTreeClassifier(max_leaf_nodes=10))

0.957746

6) Добавьте следующее условие: в дереве должно быть не более 8 листьев. Чему равна метрика F1 на отложенной выборке?

In [16]:
compute_metric(DecisionTreeClassifier(min_samples_leaf=10))

0.958333

7) Добавьте следующее условие: глубина дерева должна быть не более 4. Чему равна метрика F1 на отложенной выборке?

In [17]:
compute_metric(DecisionTreeClassifier(max_depth=4))

0.957746

8) При помощи параметра "class_weight" выставите веса объектов обратно пропорциональными их частоте в обучающей выборке. Чему стала равна метрика F1 на отложенной выборке?

In [18]:
compute_metric(DecisionTreeClassifier(class_weight="balanced"))

0.965035

9) Поменяйте критерий для построения дерева решений на 'entropy', который мы разбирали на теории. Чему стала равна метрика F1 на отложенной выборке?

In [19]:
compute_metric(DecisionTreeClassifier(criterion="entropy"))

0.958904

10) Укажите параметр min_samples_split=50. Какая получилась глубина у получившегося дерева? Укажите ее ниже.

In [20]:
compute_metric(DecisionTreeClassifier(min_samples_split=50))

0.937931

11) Вставьте в код условие, которое выполнит эвристику `min_samples_split`.

```python
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        
        self.root = None
    
    def _fit_id3(self, x, y, level):
        class_labels = np.unique(y)
        # if there is only one class in y then return leaf node
        if class_labels.size == 1:
            self.n_nodes += 1
            return Node(class_label=class_labels[0])

        # find the most informative predicate and split by them
        beta = Predicate(x, y)
        to_left, to_right = beta.split(x, y)

        # if one of arrays is empty after split then return leaf node
        if (to_left[0].size == 0 or 
            to_right[0].size == 0 or 
            level > self.max_depth or
            _______________________):
            labels, counts = np.unique(y, return_counts=True)
            class_label = labels[counts.argmax(axis=0)]
            self.n_nodes += 1
            return Node(class_label=class_label)

        node = Node()
        node.set_predicate(beta)

        node.set_left(self._fit_id3(to_left[0], to_left[1], level + 1))
        node.set_right(self._fit_id3(to_right[0], to_right[1], level + 1))

        return node
```

In [21]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        
        self.root = None
    
    def _fit_id3(self, x, y, level):
        class_labels = np.unique(y)
        # if there is only one class in y then return leaf node
        if class_labels.size == 1:
            self.n_nodes += 1
            return Node(class_label=class_labels[0])

        # find the most informative predicate and split by them
        beta = Predicate(x, y)
        to_left, to_right = beta.split(x, y)

        # if one of arrays is empty after split then return leaf node
        if (to_left[0].size == 0 or to_right[0].size == 0 or level > self.max_depth or self.min_samples_split >= 2):
            labels, counts = np.unique(y, return_counts=True)
            class_label = labels[counts.argmax(axis=0)]
            self.n_nodes += 1
            return Node(class_label=class_label)

        node = Node()
        node.set_predicate(beta)

        node.set_left(self._fit_id3(to_left[0], to_left[1], level + 1))
        node.set_right(self._fit_id3(to_right[0], to_right[1], level + 1))

        return node

12) Вставьте в код условие, которое выполнит эвристику `min_samples_leaf`.

```python
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        
        self.root = None
    
    def _fit_id3(self, x, y, level):
        class_labels = np.unique(y)
        # if there is only one class in y then return leaf node
        if class_labels.size == 1 or ________________:
            self.n_nodes += 1
            return Node(class_label=class_labels[0])

        # find the most informative predicate and split by them
        beta = Predicate(x, y)
        to_left, to_right = beta.split(x, y)

        # if one of arrays is empty after split then return leaf node
        if (to_left[0].size == 0 or 
            to_right[0].size == 0 or 
            level > self.max_depth):
            labels, counts = np.unique(y, return_counts=True)
            class_label = labels[counts.argmax(axis=0)]
            self.n_nodes += 1
            return Node(class_label=class_label)

        node = Node()
        node.set_predicate(beta)

        node.set_left(self._fit_id3(to_left[0], to_left[1], level + 1))
        node.set_right(self._fit_id3(to_right[0], to_right[1], level + 1))

        return node
```

In [ ]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        
        self.root = None
    
    def _fit_id3(self, x, y, level):
        class_labels = np.unique(y)
        # if there is only one class in y then return leaf node
        if class_labels.size == 1 or min_samples_leaf >= 1:
            self.n_nodes += 1
            return Node(class_label=class_labels[0])

        # find the most informative predicate and split by them
        beta = Predicate(x, y)
        to_left, to_right = beta.split(x, y)

        # if one of arrays is empty after split then return leaf node
        if (to_left[0].size == 0 or to_right[0].size == 0 or level > self.max_depth):
            labels, counts = np.unique(y, return_counts=True)
            class_label = labels[counts.argmax(axis=0)]
            self.n_nodes += 1
            return Node(class_label=class_label)

        node = Node()
        node.set_predicate(beta)

        node.set_left(self._fit_id3(to_left[0], to_left[1], level + 1))
        node.set_right(self._fit_id3(to_right[0], to_right[1], level + 1))

        return node

## 5.2

In [ ]:
Все задания выполняются на основе датасета UCI ML Breast Cancer Wisconsin. (https://goo.gl/U2Uwz2)  
Все признаки являются числовыми.

In [ ]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

## Задачи с проверкой кода

1) Реализовать функцию нахождения самого частого элемента в массиве (голосование)

In [35]:
def get_most_frequent_value(sequence):
    unique, counts = np.unique(sequence, return_counts=True)
    d = dict(zip(unique, counts))
    i = list(d.values()).index(max(counts))

    return list(d.keys())[i]

In [38]:
get_most_frequent_value([1,2,3,3,3,4,4])

3

2. Реализуйте функцию, выполняющую бутстреп. Ваш алгоритм должен сохранить размерность входного параметра values! В данном случае, функция должна вернуть одномерный массив

In [6]:
def bootstrap(values, new_dataset_size):
    res = np.empty((0,values.shape[1]), int)
    for i in range(new_dataset_size):
        res = np.append(res, values[np.random.choice(range(0,values.shape[0]),1)], axis = 0)
    return res

In [7]:
A = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [8]:
bootstrap(A, 5)

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [7, 8, 9],
       [4, 5, 6]])

3. Реализовать фукнцию для подсчета произвольной метрики из Scikit-Learn в режиме OOB.

In [9]:
def estimate_oob_metric(forest, metric):
    return forest.oob_score_

In [18]:
forest = RandomForestClassifier(n_estimators=100, max_depth=8, criterion='entropy', oob_score=True, random_state=42)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
f1 = f1_score(y_train, y_pred)

estimate_oob_metric(forest, f1)

0.967032967032967

## Задачи с проверкой ответа

В следующих заданих требуется так или иначе изменить параметры дерева решений и оценить результат.  
Как метрику качества будем использовать F1-score. В каждом задании необходимо поменять __только один__ параметр.  

Случайный лес — стохастический алгоритм, а значит при каждом запуске может выдавать различный результат. В дальнейших задачах __необходимо__ выставить случайность: добавить параметр `random_state=8` как аргумент при создании ансамблей.

In [19]:
def compute_metric(clf, X_train=X_train, y_train=y_train,
                   X_test=X_test, y_test=y_test):
    np.random.seed(42)
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred), 6)

Посчитаем значение метрики со стандартными параметрами.

In [21]:
forest = RandomForestClassifier(random_state=8)
compute_metric(forest)

0.965035

1) Напишите индекс самого важного признака по мнению случайного леса. Важности признаков указаны в переменной feature_importances_ случайного леса.

In [24]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
print('Важность:', forest.feature_importances_)
m = max(forest.feature_importances_)
print('Самый важный признак (индекс):', list(forest.feature_importances_).index(m))

Важность: [0.05217508 0.01589939 0.02342911 0.03752801 0.00978005 0.01579613
 0.0435446  0.1333922  0.00349372 0.00508921 0.01276953 0.00376161
 0.0045582  0.02468266 0.00425124 0.00476301 0.01040772 0.00433004
 0.00538779 0.00716034 0.06055938 0.0201828  0.18704382 0.10320636
 0.01537696 0.01515053 0.04969264 0.11202264 0.00946708 0.00509815]
Самый важный признак (индекс): 22


2) Увеличьте количество в случайном лесе до 100. Как изменилось качество? Укажите его.

In [25]:
forest = RandomForestClassifier(random_state=8, n_estimators=100)
compute_metric(forest)

0.965035

3) Выставите количество деревьев в лесе равным 100 и включите режим подсчета out-of-bag score. Укажите полученное значение метрики accuracy, округленного до 6 знака после запятой.

In [26]:
def calc_accuracy(y_true, y_pred):
    P = np.sum(y_true == 1)
    N = np.sum(y_true == 0)
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    acc = (TP + TN) / (P + N)
    
    return acc

In [27]:
forest = RandomForestClassifier(random_state=8, n_estimators=100, oob_score=True)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
acc = calc_accuracy(y_train, y_pred)
print('acc = {}%'.format(acc * 100))

acc = 100.0%


4) Выключите режим random subspaces в случайном лесе. Укажите качество.

In [28]:
forest = RandomForestClassifier(n_estimators=100, oob_score=True)
compute_metric(forest)

0.972222

5) Выключите режим бутстрапа и переобучите случайный лес. Укажите качество.

In [29]:
forest = RandomForestClassifier(n_estimators=100, bootstrap=False)
compute_metric(forest)

0.965035

6) Какая средняя глубина у деревьев, которые получаются в случайном лесе со стандартными параметрами при его обучении на этой обучающей выборке?

In [30]:
sum_depth = 0
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
for i in range(forest.n_estimators):
    sum_depth += forest.estimators_[0].tree_.max_depth
print('Средняя глубина деревьев леса =', sum_depth/forest.n_estimators)

Средняя глубина деревьев леса = 7.0
